## ДЗ №3 Двухуровневый пайплайн
#### В этой домашке вам предстоит написать с нуля двустадийную рекомендательную систему.

#### Дата выдачи: 10.03.25

#### Мягкий дедлайн: 31.03.25 23:59 MSK

#### Жесткий дедлайн: 7.04.25 23:59 MSK

### Описание
Это творческое задание, в котором вам необходимо реализовать полный цикл построения рекомендательной системы: реализовать кандидат генераторов, придумать и собрать признаки, обучить итоговый ранкер и заинференсить модели на всех пользователей.

Вам предоставляется два набора данных: `train.csv` и `test.csv`

In [ ]:
# скачиваем данные
# если из этой ячейки не получается, то вот ссылка на папку https://drive.google.com/drive/folders/1HT0Apm8Jft0VPLJtdBBUGu9s1M7vZcoJ?usp=drive_link

# !pip3 install gdown


# import gdown
# # train
# url = "https://drive.google.com/file/d/1-CcS22-UpTJeNcFlA0dVLrEQn8jnI0d-/view?usp=drive_link"
# output = 'train.csv'
# gdown.download(url, output, quiet=False)

# # test
# url = "https://drive.google.com/file/d/11iz3xDh0IIoEIBY0dyRSvByY3qfiT3BG/view?usp=drive_link"
# output = 'test.csv'
# gdown.download(url, output, quiet=False)

In [ ]:
# !pip install

In [12]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

from typing import Union
from scipy.sparse.linalg import svds
from scipy.sparse import coo_matrix, coo_array, csr_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


RANDOM_STATE = 42

np.random.seed(RANDOM_STATE)



### 1 Этап. Модели первого уровня. (max 3 балла)
В этом этапе вам необходимо разделить `train` датасет на 2 части: для обучения моделей первого уровня и для их валидации. Единственное условие для разбиения – разбивать нужно по времени. Данные для обучение будем называть `train_stage_1`, данные для валидации `valid_stage_1`. Объемы этих датасетов вы определяет самостоятельно.

Для начала нам нужно отобрать кандидатов при помощи легких моделей. Необходимо реализовать 3 типа моделей:
1. Любая эвристическая(алгоритмичная) модель на ваш выбор **(0.5 балл)**
2. Любая матричная факторизация на ваш выбор **(1 балл)**
3. Любая нейросетевая модель на ваш выбор **(1 балла)**

Не забудьте использовать скор каждой модели, как признак!



Каждая модель должна уметь:
1) для пары user_item предсказывать скор релевантности (масштаб скора не важен), важно обработать случаи, когда модель не можеn проскорить пользователя или айтем, вместо этого вернуть какое-то дефолтное значение
2) для всех пользователей вернуть top-k самых релевантных айтемов (тут вам скоры не нужны)


Дополнительно можно провести анализ кандидат генератов, измерить насколько различные айтемы они рекомендуют, например с помощью таких метрик как: [Ranked based overlap](https://github.com/changyaochen/rbo) или различные вариации [Diversity](https://github.com/MaurizioFD/RecSys2019_DeepLearning_Evaluation/blob/master/Base/Evaluation/metrics.py#L289). **(1 балл)**

In [13]:
def get_train_test_val(sample_size: int = 10000):
    df = pd.read_csv('train_part.csv')
    df_valid = pd.read_csv('test_part.csv')

    train_end = '2021-07-01'
    df_train = df[df['last_watch_dt'] < train_end].copy()
    df_test = df[df['last_watch_dt'] >= train_end].copy()

    train_users = df_train['user_id'].unique()
    train_items = df_train['item_id'].unique()

    df_test = df_test[df_test['user_id'].isin(train_users)]
    df_test = df_test[df_test['item_id'].isin(train_items)]

    unique_user_ids = df_train['user_id'].unique()

    selected_user_ids = np.random.choice(unique_user_ids, size=sample_size, replace=False)
    df_train_sample = df_train[df_train['user_id'].isin(selected_user_ids)].copy()
    df_test_sample = df_test[df_test['user_id'].isin(selected_user_ids)].copy()
    df_valid_sample = df_valid[df_valid['user_id'].isin(selected_user_ids)].copy()

    all_user_ids = np.unique(np.concatenate([df_train_sample['user_id'], df_test_sample['user_id'], df_valid_sample['user_id']]))
    all_item_ids = np.unique(np.concatenate([df_train_sample['item_id'], df_test_sample['item_id'], df_valid_sample['item_id']]))

    user_le = LabelEncoder()
    item_le = LabelEncoder()

    user_le.fit(all_user_ids)
    item_le.fit(all_item_ids)

    df_train_sample['user_id'] = user_le.transform(df_train_sample['user_id'])
    df_train_sample['item_id'] = item_le.transform(df_train_sample['item_id'])

    df_test_sample['user_id'] = user_le.transform(df_test_sample['user_id'])
    df_test_sample['item_id'] = item_le.transform(df_test_sample['item_id'])

    df_valid_sample['user_id'] = user_le.transform(df_valid_sample['user_id'])
    df_valid_sample['item_id'] = item_le.transform(df_valid_sample['item_id'])

    return df_train_sample, df_test_sample, df_valid_sample

In [14]:
df_train, df_test, df_valid = get_train_test_val()
df_train.shape, df_test.shape, df_valid.shape

((47575, 6), (16428, 6), (3480, 6))

1.1. Эвристическая модель (TopPopular)

In [15]:
class TopPopular:
    def __init__(self):
        self.item_popularity = None
        self.user_item_interactions = None

    def fit(self, df):

        self.user_item_interactions = df

        self.item_popularity = df.groupby('item_id').size().reset_index(name='popularity')

    def get_top_k_items(self, k: int = 100):
        return self.item_popularity.nlargest(k, 'popularity')['item_id'].tolist()

    def predict_relevance(self, user_id, item_id, default_value=0):
        return self.user_item_interactions[(self.user_item_interactions['user_id'] == user_id) &
                                               (self.user_item_interactions['item_id'] == item_id)]['target'].sum()

In [28]:
model = TopPopular()
model.fit(df_train)

user_id = 4338
item_id = 3552
relevance_score = model.predict_relevance(user_id, item_id)
print(f'Relevance score for user {user_id} and item {item_id}: {relevance_score}')

top_k_items = model.get_top_k_items(5)
print(f'Top 5 popular items: {top_k_items}')

Relevance score for user 4338 and item 3552: 0
Top 5 popular items: [3408, 5031, 4520, 1340, 3173]


1.2. Модель матричной факторизации (SVD)

In [22]:
class SVD_factorization():
    def __init__(self, n_singular_values: int = -1) -> None:

        self.n_singular_values = n_singular_values
        self.recs = None
        self.user_features = None
        self.item_features = None

    def _df_to_matrix(self, df: pd.DataFrame) -> np.ndarray:

        interaction_matrix = df.pivot_table(index='user_id', columns='item_id', values='target', fill_value=0)
        result = interaction_matrix.values
        return result

    def _make_svd(self, interactions: np.ndarray):

        U, S, Vt = np.linalg.svd(interactions, full_matrices=False)

        if self.n_singular_values != -1:
            U = U[:, :self.n_singular_values]
            S = S[:self.n_singular_values]
            Vt = Vt[:self.n_singular_values, :]

        self.user_features = U
        self.item_features = Vt.T
        return U, S, Vt

    def fit(self, df_train: pd.DataFrame):

        interactions = self._df_to_matrix(df_train)
        U, S, Vt = self._make_svd(interactions)

        self._calculate_recommendations(interactions)

    def _calculate_recommendations(self, interactions):

        n_users, n_items = interactions.shape
        relevance_scores = np.zeros((n_users, n_items))

        for user in range(n_users):
            interacted_items = np.where(interactions[user] > 0)[0]

            user_features = self.user_features[user]
            item_features = self.item_features

            relevance_scores[user] = np.dot(user_features, item_features.T)

            if np.isnan(relevance_scores[user]).any():
                relevance_scores[user] = np.nan_to_num(relevance_scores[user], nan=0.0)

            relevance_scores[user, interacted_items] = -np.inf

        self.recs = relevance_scores

    def predict_relevance(self, user_id: int, item_id: int) -> float:

        if self.user_features is None or self.item_features is None:
            raise ValueError("Model is not fitted yet.")

        user_features = self.user_features[user_id]
        item_features = self.item_features[item_id]

        relevance_score = np.dot(user_features, item_features)

        if np.isnan(relevance_score):
            return 0.0

        return float(relevance_score)

    def get_top_k_recommendations(self, user_id: int, top_k: int) -> list:

        if self.recs is None:
            raise ValueError("Recommendations are not calculated yet.")

        relevance_scores = self.recs[user_id]
        top_k_indices = np.argsort(relevance_scores)[-top_k:][::-1]

        return top_k_indices.tolist()

In [23]:
mf = SVD_factorization()

mf.fit(df_train)

In [24]:
mf.get_top_k_recommendations(user_id=4338, top_k=10)

[1281, 3546, 495, 3437, 1328, 876, 1892, 1926, 4100, 987]

In [27]:
mf.predict_relevance(4338, 3552)

7.565110139655319e-05

1.3. Модель NCF (колаборативная фильтрация на основе нейровнной сети)

In [29]:
class NCFModel:
    def __init__(self, data, embedding_dim=8, device='cpu'):
        self.device = device
        self.data = data
        self.embedding_dim = embedding_dim

        self.prepare_data()

        self.model = self.NCFRecommender(self.num_users, self.num_items, self.embedding_dim).to(self.device)
        self.criterion = nn.BCELoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.001)

    class NCFRecommender(nn.Module):
        def __init__(self, num_users, num_items, embedding_dim):
            super(NCFModel.NCFRecommender, self).__init__()
            self.user_embedding = nn.Embedding(num_users, embedding_dim)
            self.item_embedding = nn.Embedding(num_items, embedding_dim)
            self.fc = nn.Sequential(
                nn.Linear(embedding_dim * 2, 64),
                nn.ReLU(),
                nn.Linear(64, 32),
                nn.ReLU(),
                nn.Linear(32, 1),
                nn.Sigmoid()
            )

        def forward(self, user_index, item_index):
            user_vec = self.user_embedding(user_index)
            item_vec = self.item_embedding(item_index)
            concat = torch.cat([user_vec, item_vec], dim=-1)
            return self.fc(concat)

    def prepare_data(self):
        user_ids = self.data['user_id'].unique()
        item_ids = self.data['item_id'].unique()

        self.user_id_mapping = {id: idx for idx, id in enumerate(user_ids)}
        self.item_id_mapping = {id: idx for idx, id in enumerate(item_ids)}

        self.data['user_index'] = self.data['user_id'].map(self.user_id_mapping)
        self.data['item_index'] = self.data['item_id'].map(self.item_id_mapping)

        X = self.data[['user_index', 'item_index']]
        y = self.data['target']

        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        self.X_train_tensor = torch.tensor(self.X_train.values, dtype=torch.long).to(self.device)
        self.y_train_tensor = torch.tensor(self.y_train.values, dtype=torch.float32).to(self.device)

        self.num_users = len(user_ids)
        self.num_items = len(item_ids)

    def fit(self, epochs=10, batch_size=32):
        self.model.train()
        dataset = torch.utils.data.TensorDataset(self.X_train_tensor[:, 0], self.X_train_tensor[:, 1], self.y_train_tensor)
        dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

        for epoch in range(epochs):
            for user_index, item_index, target in dataloader:
                self.optimizer.zero_grad()
                output = self.model(user_index, item_index)
                loss = self.criterion(output.squeeze(), target)
                loss.backward()
                self.optimizer.step()
            print(f'Epoch {epoch + 1}/{epochs}, Loss: {loss.item():.4f}')

    def predict_relevance(self, user_index, item_index):
        self.model.eval()
        with torch.no_grad():
            user_index_tensor = torch.tensor(user_index, dtype=torch.long).to(self.device)
            item_index_tensor = torch.tensor(item_index, dtype=torch.long).to(self.device)
            return self.model(user_index_tensor, item_index_tensor).cpu().numpy()

    def get_top_k_recommendations(self, user_index, item_indices, k=5):

        scores = self.predict_relevance([user_index] * len(item_indices), item_indices)

        top_k_indices = scores.flatten().argsort()[-k:][::-1]

        return [item_indices[i] for i in top_k_indices]

In [30]:
ncf = NCFModel(df_train, embedding_dim=8, device='cuda' if torch.cuda.is_available() else 'cpu')
ncf.fit(epochs=10, batch_size=2)

Epoch 1/10, Loss: 0.5879
Epoch 2/10, Loss: 0.4578
Epoch 3/10, Loss: 0.5746
Epoch 4/10, Loss: 1.0551
Epoch 5/10, Loss: 0.4241
Epoch 6/10, Loss: 0.6885
Epoch 7/10, Loss: 0.0746
Epoch 8/10, Loss: 0.3132
Epoch 9/10, Loss: 1.2987
Epoch 10/10, Loss: 0.4426


In [32]:
user_id = 4338
item_id = 3552
user_index = ncf.user_id_mapping[user_id]
item_index = ncf.item_id_mapping[item_id]

predicted_score = ncf.predict_relevance([user_index], [item_index])
print(f'relevance score for user {user_id} and item {item_id}: {predicted_score[0][0]}')

item_indices = list(ncf.item_id_mapping.values())
top_k_recommendations = ncf.get_top_k_recommendations(user_index, item_indices, k=3)
print(f'top K recommendations for user {user_id}: {top_k_recommendations}')

relevance score for user 4338 and item 3552: 0.7109264135360718
top K recommendations for user 4338: [500, 800, 2295]



### 2 Этап. Генерация и сборка признаков. (max 2 балла)
Необходимо собрать минимум 10 осмысленных (`np.radndom.rand()` не подойдет) признаков, при этом:
1. 2 должны относиться только к сущности "пользователь" (например средний % просмотра фильмов у этой возрастной категории)
2. 2 должны относиться только к сущности "айтем" (например средний средний % просмотра данного фильма)
3. 6 признаков, которые показывают связь пользователя и айтема (например средний % просмотра фильмов с данным актером (айтем) у пользователей с таким же полом (пользователь)).

### ВАЖНО!  

1. **В датасете есть колонка `watched_prct`. Ее можно использовать для генерации признаков (например сколько пользователь в среднем смотрит фильмы), но нельзя подавать в модель, как отдельную фичу, потому что она напрямую связана с target.**
2. **Все признаки должны быть собраны без дата лика, то есть если пользователь посмотрел фильм 10 августа, то признаки мы можем считать только на данных до 9 августа включительно.**


### Разбалловка
Обучение ранкера будет проходить на `valid_stage_1`, как  раз на которой мы валидировали модели, а тестировать на `test`. Поэтому есть 2 варианта сборки признаков, **реализовать нужно только 1 из них:**
1. Для обучения собираем признаки на первый день `valid_stage_1`, а для теста на первый день `test`. Например, если `valid_stage_1` начинается 5 сентября, то все признаки мы можем собирать только по 4 сентября включительно. **(1 балл)**
2. Признаки будем собирать честно на каждый день, то есть на 5 сентября собираем с начала до 4, на 6 сентября с начала до 5 и т.д. **(2 балла)**

In [ ]:
item_features = pd.read_csv('items.csv')
user_features = pd.read_csv('users.csv')

In [ ]:
user_features.head()

In [ ]:
train_df_with_features = # YOUR CODE IS HERE
test_df_with_features = # YOUR CODE IS HERE


### 3 Этап. Обучение финального ранкера (max 2 балла)
Собрав все признаки из этапа 2, добавив скоры моделей из этапа 1 для каждой пары пользователь-айтем (где это возможно), пришло время обучать ранкер. В качестве ранкера можно использовать либо [xgboost](https://xgboost.readthedocs.io/en/stable/) или [catboost](https://catboost.ai/). Обучать можно как `Classfier`, так и `Ranker`, выбираем то, что лучше сработает. Обучение ранкера будет проходить на `valid_stage_1`, как  раз на которой мы валидировали модели, а тестировать на `test`, которую мы до сих пор не трогали.  Заметьте, что у нас в тесте есть холодные пользователи – те, кого не было в train и активные – те, кто был в train. Возможно их стоит обработать по отдельности (а может и нет).  
(1 балл)

После получения лучшей модели надо посмотреть на важность признаков и [shap values](https://shap.readthedocs.io/en/latest/index.html), чтобы:
1. Интерпритировать признаки, которые вы собрали, насколько они полезные
2. Проверить наличие ликов – если важность фичи в 100 раз больше, чем у всех остальных, то явно что-то не то  

(1 балл)






In [ ]:
# YOUR FIT PREDICT CODE HERE
model.fit()
model.predict()


### 4 Этап. Инференс лучшего ранкера (max 3 балла)

Теперь мы хотим построить рекомендации "на завтра", для этого нам нужно:

1. Обучить модели первого уровня на всех (train+test) данных (0.5 балла)
2. Для каждой модели первого уровня для каждого пользователя сгененировать N кандидатов (0.5 балла)
3. "Склеить" всех кандидатов для каждого пользователя (дубли выкинуть), посчитать скоры от всех моделей (0.5 балла)
4. Собрать фичи для ваших кандидатов (теперь можем считать признаки на всех данных) (0.5 балла)
5. Проскорить всех кандидатов бустингом и оставить k лучших (0.5 балла)
6. Посчитать разнообразие(Diversity) и построить график от Diversity(k) (0.5 балла)


Все гиперпараметры (N, k) определяете только Вы!

In [ ]:
# YOUR CODE HERE